In [134]:
'''
    Load model and split it
        1. layer by layer
        2. [TODO] vertically 
'''

'\n    Load model and split it\n        1. layer by layer\n        2. [TODO] vertically \n'

In [135]:
from source.core.engine import MoP
import source.core.run_partition as run_p
from os import environ
from source.utils.dataset import *
from source.utils.misc import *
from source.utils.split_network import *

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from source.models import resnet
from source.core.split_manager import SplitManager

import torch.nn.functional as F

import numpy as np

from source.utils import io
from source.utils import testers
from source.core import engine
import json
import itertools

from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchsummary import summary

from torch.autograd import Variable

import time

In [136]:
# Set the model setup

# setup = 'resnet18'
setup = 'escnet'
# setup = 'flashnet'

yaml_version = 'v1'

In [137]:
# setup config

if setup == 'resnet18':
    dataset='cifar10'
    load_model = f"cifar10-resnet18-kernel-np{yaml_version}-pr0.75-lcm0.001.pt"
elif setup == 'escnet':
    dataset='esc'
    load_model = f"esc-escnet-kernel-np{yaml_version}-pr0.75-lcm0.001.pt"
elif setup == 'flashnet':
    dataset='flash'
    load_model = f"flash-flashnet-kernel-np{yaml_version}-pr0.75-lcm0.001.pt"

environ["config"] = f"config/{dataset}.yaml"
configs = run_p.main()
configs["device"] = "cpu"
configs['load_model'] = load_model

if setup == 'resnet18':
    configs["num_partition"] = f'config/resnet18-{yaml_version}.yaml'
elif setup == 'escnet':
    configs["num_partition"] = f'config/escnet-{yaml_version}.yaml'
elif setup == 'flashnet':
    configs["num_partition"] = f'config/flashnet.yaml'

device :  
model :  EscFusion
data_code :  esc
num_classes :  2
model_file :  test.pt
epochs :  0
batch_size :  32
optimizer :  adam
lr_scheduler :  cosine
learning_rate :  0.1
seed :  1234
sparsity_type :  kernel
prune_ratio :  1
admm :  True
admm_epochs :  300
rho :  0.01
multi_rho :  True
retrain_bs :  128
retrain_lr :  0.005
retrain_ep :  50
retrain_opt :  adam
xentropy_weight :  1.0
warmup :  False
warmup_lr :  0.01
warmup_epochs :  10
mix_up :  True
alpha :  0.3
smooth :  False
smooth_eps :  0
save_last_model_only :  False
num_partition :  1
layer_type :  regular
bn_type :  regular
par_first_layer :  False
comm_outsize :  False
lambda_comm :  0
lambda_comp :  0
distill_model :  
distill_loss :  kl
distill_temp :  30
distill_alpha :  1


In [138]:
# load data and load or train model
model = get_model_from_code(configs).to(configs['device']) # grabs model architecture from ./source/models/escnet.py
print(model)

ResLike(
  (conv1): Conv2d(15, 32, kernel_size=(7, 7), stride=(1, 1), padding=same)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(3

In [139]:
# load weights into full model
state_dict = torch.load(io.get_model_path_split("{}".format(configs["load_model"])), map_location=configs['device'])
model = io.load_state_dict(model, 
                    state_dict['model_state_dict'] if 'model_state_dict' in state_dict 
                    else state_dict['state_dict'] if 'state_dict' in state_dict else state_dict,)


In [140]:
'''
    add partitions and communications to configs
'''

# gets random test input (with correct size)
input_var = get_input_from_code(configs)

# Config partitions and prune_ratio
configs = engine.partition_generator(configs, model)
            
# Compute output size of each layer
configs['partition'] = engine.featuremap_summary(model, configs['partition'], input_var)
        
# Setup communication costs
configs['comm_costs'] = engine.set_communication_cost(model, configs['partition'],)


# split model general parameters

# make copies of model per machine
num_machines = max(configs['partition']['bn_partition']) # TODO: double check this makes sense
model_machines = [model]*num_machines

layer_names_fx =  get_graph_node_names(model)[1]
total_layers_fx = len(layer_names_fx)
print(f"Total layers: {total_layers_fx}")

split_module_names = list(configs['partition'].keys())

print(layer_names_fx)
print('num_machines:', num_machines)


num_partition: {'inputs': 5, 'conv1.weight': 5, 'layer1.0.conv1.weight': 5, 'layer1.0.conv2.weight': 5, 'layer2.0.conv1.weight': 5, 'layer2.0.conv2.weight': 5, 'hidden1.weight': 5, 'hidden2.weight': 5, 'hidden3.weight': 5, 'conv2.weight': 5}
ratio_partition: {'inputs': [1, 1, 1, 1, 1], 'conv1.weight': [1, 1, 1, 1, 1], 'layer1.0.conv1.weight': [1, 1, 1, 1, 1], 'layer1.0.conv2.weight': [1, 1, 1, 1, 1], 'layer2.0.conv1.weight': [1, 1, 1, 1, 1], 'layer2.0.conv2.weight': [1, 1, 1, 1, 1], 'hidden1.weight': [1, 1, 1, 1, 1], 'hidden2.weight': [1, 1, 1, 1, 1], 'hidden3.weight': [1, 1, 1, 1, 1], 'conv2.weight': [1, 1, 1, 1, 1]}
map_partition: {'conv1.weight': [[0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]], 'layer1.0.conv1.weight': [[0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]], 'layer1.0.conv2.weight': [[0, 1, 1, 1, 1], [1, 0, 1, 1, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0]], 'layer2.0.conv1.weight': [[0, 1

In [141]:
''' Prep Inout for Mock Run version 2 (fx)'''

# TODO: reduce size of communicated tensors to only what is necessary 
# TODO: also check bias for nonzero
# TODO: come up with more general scheme to handle residual layers

# channel_id == INPUTS
# filter_id  == OUTPUTS

# try greater precision
#torch.set_default_tensor_type(torch.DoubleTensor)
#torch.set_default_dtype(torch.float64)

# setup input 
N_batch = 1
torch.manual_seed(0)

if setup == 'resnet18':
    input_tensor = torch.rand(N_batch, 3, 32, 32, dtype=torch.float64, device=torch.device(configs['device'])) # 1k images, 3 channels, 32x32 image (cifar10)
    input_tensor = tuple([input_tensor])
    input_tensor = torch.cat(input_tensor, dim=1)
elif setup == 'escnet':
    input_tensor = torch.rand(N_batch, 3, 266, 320, dtype=torch.float64, device=torch.device(configs['device']))
    # input_tensor = torch.cat([input_tensor]*5, dim=1)
    input_tensor = tuple([input_tensor]*5)
    input_tensor = torch.cat(input_tensor, dim=1)
elif setup == 'flashnet':
    input_tensor = torch.rand(N_batch, 2, 10, 10, dtype=torch.float64, device=torch.device(configs['device']))


# if configs['data_code'] == 'flash':
#     input_shape = [(2, 1),
#                 #    (90, 160, 3),
#                     (360, 640, 3),
#                     (20, 20, 20),]
# elif configs['data_code'] == 'esc':
#     input_shape = [(3, 266, 320) for _ in range(5)]
# else:
#     input_shape = [(3, 32, 32)]

# input_np = (np.random.uniform(0, 1, (1,)+x) for x in input_shape)
# input_tensor = tuple(Variable(torch.FloatTensor(x), requires_grad=False).to(configs["device"]) for x in input_np)
# # input_tensor = torch.cat(input_tensor, dim=1)

print(input_tensor.shape)
print(input_tensor[0].shape)
print(len(input_tensor))
# print(f'input_tensor: {input_tensor}')

model = model.type(torch.float64)

# make SplitManagers for split model execution 
configs['dtype'] = 'float64'
split_managers = [SplitManager]*num_machines
for i in range(num_machines):
    split_managers[i] = SplitManager(configs, i, num_machines, input_tensor)

# broadcast input_tensor to different machines
# TODO: find a better datastructure for this
#input = np.empty((num_machines, num_machines), dtype=torch.Tensor)
input = [None]*num_machines
input = [input[:] for i in range(num_machines)]
for imach in range(num_machines):
    input[imach][imach] = input_tensor

# print(f'input: {input}')

residual_block_start, residual_connection_start, residual_block_end = get_residual_block_indexes(model)

# put models into eval mode and on device
model.eval()
model.to(configs['device'])

# set index to execute vertical splitting up to 
exec_to_layer = total_layers_fx-1

# get true output at each layer
# WARNING: bn layers are initialized differently between model in this notebook and model in split manager. The moving mean and variance fields do not match HOWEVER the weights and biases are the same
#horz_output, size_LUT = get_output_at_each_layer(model, input_tensor) 
horz_output, size_LUT = get_output_at_each_layer(split_managers[0].model, input_tensor)

BREAK_LOOP = 0 # break loop when output differs

'''
    mock run through inference using split models 
'''

# timing
split_execution_start_time = time.time()
layer_completion_time_stamp = {}
layer_execution_duration = {}

# make inference 
with torch.no_grad():
    residual_input = {} # use this to keep track of inputs stored in machine memory for residule layers

    # iterate through layers 1 module at a time 
    for imodule in range(exec_to_layer+1):#range(num_total_modules): # 16 <=> layer_1 block 

        # initialize output for ilayer
        #output = np.empty((num_machines, num_machines), dtype=torch.Tensor) # square list indexed as: output[destination/RX machine][origin/TX machine]
        # TODO: find a better datastructure for this 

        output = [None]*num_machines
        output = [output[:] for i in range(num_machines)]

        # print(f'output: {output}')

        # DEBUG
        full_input = combine_all_inputs(input, num_machines)

        print(f'Executing module {imodule}: {layer_names_fx[imodule]}')

        # iterate through each machine (done in parallel later)
        for imach in range(num_machines):
            print(f'\tExecuting on machine {imach}')
            # print(f'curr_input: {curr_input}')
            # print(f'input: {input}')
            # collect communication inputs if necessary 
            if not imodule == 0 and ('conv' in layer_names_fx[imodule-1] or 'linear' in layer_names_fx[imodule-1] or 'shortcut.1' in layer_names_fx[imodule]): # TODO: this is very hacky, needs to be generalized. The issue is ID'ing conv layers in shortcut blocks
                curr_input = combine_inputs(input, num_machines, imach)
                # print(f'ilk if e girdi, curr_input: {curr_input}')
            else:
                curr_input = input[imach][imach]
                # print(f'ikinci elif e girdi, curr_input: {curr_input}')

            # print(f'len curr_input: {len(curr_input)}')
            
            out_tensor, do_comms = split_managers[imach].execute_split_layer(curr_input, imodule)
            # print(f'out_tensor: {out_tensor}')
            if not do_comms:
                # update output to current machine and continue
                if torch.is_tensor(out_tensor):
                    # sometimes out_tensor is None
                    # input is sent to all machines for 1st layer execution even though not all machines need to compute 
                    # Output from machine is None in this case TODO: fix where inputs are sent 
                    output[imach][imach] = out_tensor
                    # print(f'output: {output}')
                continue
            # print(f'out_tensor: {out_tensor}')
            # END SplitManager execute split_layer

            print(f'\t\t Output tensor shape : {out_tensor.shape}')

            # debug
            nonzero_out_tensor = torch.unique(torch.nonzero(out_tensor, as_tuple=True)[1])

            # look at which C_out need to be computed and sent
            #nonzero_Cout = torch.unique(torch.nonzero(split_layer.weight, as_tuple=True)[0]) # find nonzero dimensions in output channels
            nonzero_Cout = get_nonzero_channels(out_tensor)

            # prep communications by populating output
            out_channel_array = torch.arange(out_tensor.shape[1])
            for rx_mach in range(num_machines):
                # only add to output if communication is necessary 

                # Get output channels for current rx machine? TODO: consider removing, this just maps C_out's to machine
                #output_channels = torch.tensor(configs['partition'][][rx_mach],
                #        device=torch.device(configs['device']))
                output_channels = torch.tensor(split_managers[imach].output_channel_map[rx_mach],
                        device=torch.device(configs['device']))

                # TODO: is there a faster way to do this? Consider putting larger array 1st... just not sure which one that'd be
                nonzero_out_channels = nonzero_Cout[torch.isin(nonzero_Cout, output_channels)]
                if nonzero_out_channels.nelement() > 0:
                        communication_mask = torch.isin(out_channel_array, nonzero_out_channels)

                        # TODO: this is inefficient, redo. Probbably need to send a tensor and some info what output channels are being sent
                        tmp_out = torch.zeros(out_tensor.shape, dtype= split_managers[imach].dtype) 
                        if imodule == total_layers_fx-1 or 'hidden' in layer_names_fx[imodule]:
                                tmp_out[:,communication_mask] = out_tensor[:,communication_mask]
                        else:
                                tmp_out[:,communication_mask,:,:] = out_tensor[:,communication_mask,:,:]
                        output[rx_mach][imach] = tmp_out

                        # debug
                        print(f'\t\t sending C_out {nonzero_out_channels} to machine {rx_mach}')

        # send to next layer  
        input = output
        print(f'Finished execution of layer {imodule}')

        # update timing
        layer_completion_time_stamp[layer_names_fx[imodule]] = time.time()
        if imodule > 0:
            layer_execution_duration[layer_names_fx[imodule]] = layer_completion_time_stamp[layer_names_fx[imodule]] - layer_completion_time_stamp[layer_names_fx[imodule-1]] 
        else:
            layer_execution_duration[layer_names_fx[imodule]] = layer_completion_time_stamp[layer_names_fx[imodule]] - split_execution_start_time

        # check output at end of each layer to see if fit matches 
        tmp_output = input 
        # print(f'tmp_output = {tmp_output}')
        need_to_init  = True
        for rx_mach in range(num_machines):
                for tx_mach in range(num_machines):
                        if not tmp_output[rx_mach][tx_mach] == None:
                                if need_to_init:
                                        vert_output = tmp_output[rx_mach][tx_mach]
                                        need_to_init = False
                                else:
                                        # TODO: += causes assignment issues, switched to x = x+y which might be more more inefficent memory wise ... 
                                        vert_output = vert_output + tmp_output[rx_mach][tx_mach] 
                                        #nz_channels = get_nonzero_channels(vert_output)
                                        #print(f'({rx_mach},{tx_mach}) {nz_channels}')
        
        if imodule == total_layers_fx-1:
            # apply bias
            # TODO: assumes Linear layer is final layer and bias can be handled as final step 
            vert_output = vert_output + get_current_module(model, imodule).bias
            
            # final execution time
            tot_split_execution_time = time.time() - split_execution_start_time
            print(f'\n\n############################# FINAL EXECUTION TIME {tot_split_execution_time} [seconds] #############################\n\n')

        truth_output = horz_output[layer_names_fx[imodule]]
        # if 'x' == layer_names_fx[imodule] or '_x' == layer_names_fx[imodule] or 'getitem' == layer_names_fx[imodule] or 'getitem_1' == layer_names_fx[imodule] or 'getitem_2' == layer_names_fx[imodule] or 'getitem_3' == layer_names_fx[imodule] or 'getitem_4' == layer_names_fx[imodule] or 'cat' == layer_names_fx[imodule]:
        if 'x' == layer_names_fx[imodule] or '_x' in layer_names_fx[imodule]:    
            print(f'Input layer. Skipping comparison')
        elif torch.is_tensor(truth_output):
            max_diff, max_by_Cout = compare_outputs(vert_output, truth_output)
            if max_diff > 0.1:
                pass
                # BREAK_LOOP = 1 
        else:
            print(f'Horizontal output is {type(truth_output)}. Skipping comparison')
        print()

        if BREAK_LOOP:
            break


torch.Size([1, 15, 266, 320])
torch.Size([15, 266, 320])
1


False
False
False
False
False
False
False
False
False
False
False
False
Executing module 0: x
	Executing on machine 0
		-model input layer.. skipping
	Executing on machine 1
		-model input layer.. skipping
	Executing on machine 2
		-model input layer.. skipping
	Executing on machine 3
		-model input layer.. skipping
	Executing on machine 4
		-model input layer.. skipping
Finished execution of layer 0
Input layer. Skipping comparison

Executing module 1: conv1
	Executing on machine 0
self_residual_block_start: [ 5. 21.]
split_param_name: conv1.weight
self.split_module_names: ['bn_partition', 'conv1.weight', 'conv2.weight', 'layer1.0.conv1.weight', 'layer1.0.conv2.weight', 'layer2.0.conv1.weight', 'layer2.0.conv2.weight', 'hidden1.weight', 'hidden2.weight', 'hidden3.weight']
		 Output tensor shape : torch.Size([1, 32, 266, 320])
		 sending C_out tensor([0, 1, 2, 3, 4, 5]) to machine 0
		 sending C_out tensor([12]) to machine 2
		 sending C_out tensor([19, 20, 22]) to machine 3
		 sending

In [142]:
'''
    Conv1 layer test -- should we expect some difference, or exactly 0 in the output when we split the model?
'''

# DIFFERENCE SHOULD BE 0 NOT 1E-7

N_in = 1
split_1 = nn.Conv2d(N_in,
            model.conv1.weight.shape[0], # TODO does this need to be an int? (currently tensor)
            kernel_size= model.conv1.kernel_size,
            stride=model.conv1.stride,
            padding=model.conv1.padding, 
            bias=False) # TODO: add bias during input collecting step on next layer 
split_1.weight = torch.nn.Parameter(model.conv1.weight.index_select(1, torch.tensor([0])))  
out_split1 = split_1(input_tensor.index_select(1, torch.tensor([0])))

split_2 = split_1
split_2.weight = torch.nn.Parameter(model.conv1.weight.index_select(1, torch.tensor([1])))  
out_split2 = split_2(input_tensor.index_select(1, torch.tensor([1])))

split_3 = split_1
split_3.weight = torch.nn.Parameter(model.conv1.weight.index_select(1, torch.tensor([2])))  
out_split3 = split_3(input_tensor.index_select(1, torch.tensor([2])))

split_out = torch.add(torch.add(out_split1, out_split2), out_split3)
full_out = model.conv1(input_tensor)

diff_output = torch.abs(full_out - split_out)
max_diff = torch.max(diff_output)
max_diff.sci_mode = True
print(max_diff)

tensor(7.4249, dtype=torch.float64, grad_fn=<MaxBackward1>)


In [143]:
'''  
    Inspect I/O of single layer
'''

t = torch.ones((1,2,2,2), dtype=torch.float32) # (batch, in channel, H, W)
w = torch.ones((1,2,2,2), dtype=torch.float32) # (out channels, in channels, H, W)
w[0,0,0,0] = 1e-10
w[0,0,0,1] = 1e-10
w[0,1,0,0] = 1e-10

full_conv = torch.nn.Conv2d(2,1,kernel_size=(2,2), bias=False, stride=(1),dtype=torch.float32)
full_conv.weight = torch.nn.Parameter(w)
conv1 = torch.nn.Conv2d(1,1,kernel_size=(2,2), bias=False, stride=1, dtype=torch.float32)
conv1.weight =torch.nn.Parameter( w[:,0:1,:,:])
conv2 = torch.nn.Conv2d(1,1,kernel_size=(2,2), bias=False, stride=1,dtype=torch.float32)
conv2.weight = torch.nn.Parameter(w[:,1:2,:,:])

full_conv.eval()
conv1.eval()
conv2.eval()

with torch.no_grad():
    full_out = full_conv(t)
    split_out =  conv2(t[0,1:2,:,:]) + conv1(t[:,0:1,:,:])

diff = torch.abs(full_out - split_out)

torch.nonzero(diff)
#print()


tensor([], size=(0, 4), dtype=torch.int64)

In [144]:
# test bn1
#estimate = -bn1.running_mean[1]/torch.sqrt(bn1.running_var[1] + bn1.eps)*bn1.weight[1] + bn1.bias[1]
#estimate_split = -split_layer.running_mean[1]/torch.sqrt(split_layer.running_var[1] + split_layer.eps)*split_layer.weight[1] + split_layer.bias[1]

# running estimates are different 
#bn1.running_mean[1] - split_layer.running_mean[1]
#bn1.running_var[1] - split_layer.running_var[1] 
#bn1.weight[1] - split_layer.weight[1]
#bn1.eps - split_layer.eps
#bn1.bias[1] - split_layer.bias[1]